In [75]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import multiprocessing as mp
import pandas as pd
import scipy.io as sio

In [76]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
print('Loading and Visualizing Data ...\n')
mat_contents = sio.loadmat('ex4data1.mat')
X = mat_contents['X']
y = mat_contents['y']
m = len(y)
print(m)

rand_indices = np.random.permutation(m)
print(rand_indices)
rand_indices = rand_indices.reshape(-1, 1)
sel = X[rand_indices[0:100, :]]



Loading and Visualizing Data ...

5000
[3625 1503  827 ...  720  541  263]


In [77]:
def displayData(X):
    fig, ax = plt.subplots(10,10,sharex=True,sharey=True)
    img_num = 0
    for i in range(10):
        for j in range(10):
            # Convert column vector into 20x20 pixel matrix
            # You have to transpose to display correctly
            img = X[img_num,:].reshape(20,20).T
            ax[i][j].imshow(img,cmap='gray')
            img_num += 1

    return (fig, ax)
    
#figure, ax = displayData(sel)
#figure.show()

In [78]:
print('\nLoading Saved Neural Network Parameters ...\n')
nn_contents = sio.loadmat('ex4weights.mat')

Theta1 = nn_contents['Theta1']
Theta2 = nn_contents['Theta2']

nn_params = np.concatenate((Theta1.reshape(Theta1.size, 1, order='F'), Theta2.reshape(Theta2.size, 1, order='F')), axis=0)# [Theta1(:) ; Theta2(:)];
print(Theta1.shape)
print(Theta2.shape)
print(Theta1[0][1])
print(Theta2[0][1])


Loading Saved Neural Network Parameters ...

(25, 401)
(10, 26)
-1.0562416310683364e-08
-1.2124449845935756


In [79]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z));

In [80]:
def sigmoidGradient(z):
    return sigmoid(z) * (1-sigmoid(z))

In [98]:
def nnCostFunction(nn_params_l, input_layer_size_l, hidden_layer_size_l, num_labels_l, X_l, y_l, lambda_l):
    #print(input_layer_size_l)
    #print(hidden_layer_size_l)
    #Theta1 = reshape(nn_params_l(1:hidden_layer_size_l * (input_layer_size_l + 1)), hidden_layer_size_l, (input_layer_size_l + 1));
    Theta1_l = nn_params_l[0:(hidden_layer_size_l * (input_layer_size_l + 1))].reshape(input_layer_size_l + 1, hidden_layer_size_l).T
    Theta2_l = nn_params_l[(Theta1_l.size):nn_params_l.size].reshape(hidden_layer_size_l + 1, num_labels_l).T
    #print(Theta1_l[0][1])
    #print(Theta2_l[0][1])
    print('\nFeedforward Using Neural Network ...\n')
    
    m = X_l.shape[0] #5000X400    
    X_l = np.c_[np.ones((m, 1)), X_l] #5000X401
    z1 = X_l.dot(Theta1.T)
    a1 = sigmoid(z1)
    a1 = np.c_[np.ones((m, 1)), a1]
    
    z2=  a1.dot(Theta2.T)

    h = sigmoid(z2)
    #print(h[: 2])
    J = 0
    for k in range(1, num_labels_l + 1):
        yk = (y==k) * 1
        J = J - (1/m) * np.sum(yk * np.log(h.T[k-1:k].T) + (1-yk) * np.log(1-h.T[k-1:k].T))    
    #print(J)
    
    rtheta1 = np.sum(np.sum(np.square(Theta1_l[:,1:])))
    rtheta2 = np.sum(np.sum(np.square(Theta2_l[:,1:])))
    bias = lambda_l/(2*m)
    
    J= J + (bias * (rtheta1+rtheta2))
    Del1, Del2 = 0, 0
    
    for t in range(m): 
        A1 = X_l[t,:].T.reshape(-1, 1) # all columns with one row at a time
        Z2 = Theta1_l.dot(A1)
        A2 = np.concatenate((np.c_[np.array([1])], sigmoid(Z2).reshape(-1,1)))
        Z3 = Theta2.dot(A2)
        H = sigmoid(Z3)
        actual = y[t,:].reshape(-1,1)
        yk = np.zeros((num_labels,1))
        yk[actual - 1] = 1
        del3 = H - yk;
        del2 = (Theta2_l[:,1:].T.dot(del3)) * sigmoidGradient(Z2).reshape(-1, 1)
        #print(A2.shape)
        Del1 = Del1 + del2.dot(A1.T)
        Del2 = Del2 + del3.dot(A2.T)
    
lambda_val = 0

J = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_val)



Feedforward Using Neural Network ...



In [ ]:
(Theta1.size + 1)
nn_params.size

In [ ]:
a = np.random.rand(4, 4)


In [ ]:
a

In [ ]:
#print(a[:,2:3])
a[3,:]

In [ ]:
(hidden_layer_size * (input_layer_size + 1))